<a href="https://colab.research.google.com/github/sergeychuvakin/random_scripts/blob/master/cnn_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# то что долго нужно ждать (минут 10-15)
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
!unzip wiki.en.zip
!wget https://github.com/sergeychuvakin/random_scripts/raw/master/fin3.pickle
!pip install fasttext
!rm -rf wiki.en.zip
!rm -rf wiki.en.vec

import fasttext
model_ft = fasttext.load_model('wiki.en.bin')

import nltk 
nltk.download("all")

--2020-08-14 09:22:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  27.5MB/s    in 3m 50s  

2020-08-14 09:26:49 (42.9 MB/s) - ‘wiki.en.zip’ saved [10356881291/10356881291]

Archive:  wiki.en.zip
  inflating: wiki.en.vec             
  inflating: wiki.en.bin             
--2020-08-14 09:33:23--  https://github.com/sergeychuvakin/random_scripts/raw/master/fin3.pickle
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sergeychuvakin/

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [2]:
import torch
import pandas as pd 
import numpy as np 
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.model_selection import train_test_split
import gc

In [3]:
import pickle

with open('fin3.pickle', 'rb') as f: 
    df = pickle.load(f)

In [4]:
texts = df['text']
labels = df['_class']
word2idx = {}
del df
gc.collect()

0

In [5]:
word2idx['<pad>'] = 0
word2idx['<unk>'] = 1
idx = 2
tokens = []
max_len = 0
for sent in texts:
    _sent = word_tokenize(sent)
    tokens.append(_sent)
    max_len = max(len(_sent), max_len)
    for word in _sent:
        if word not in word2idx:
            word2idx[word] = idx
            idx += 1

In [6]:
vecs = []
for i in tokens:
    i.extend(['<pad>'] * (max_len - len(i)))
    vecs.append([word2idx[word] for word in i ])

In [7]:
emb = torch.tensor([model_ft.get_word_vector(i) for i in word2idx.keys()])

In [8]:
rule = dict(enumerate(set(labels)))
rule = {ii:i for i, ii in rule.items()}

labels = [rule[i] for i in labels]

In [9]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    vecs, labels, test_size=0.1, random_state=42)

    
train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

## обертка датасета (провила подачи на обучение)
train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data) # как выбираются батчи
train_dataloader = DataLoader(train_data, sampler=train_sampler)

# Create DataLoader for validation data
val_data = TensorDataset(val_inputs, val_labels)
val_sampler = SequentialSampler(val_data) # как подаются
val_dataloader = DataLoader(val_data, sampler=val_sampler)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class CNN(nn.Module):
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 num_classes=len(set(labels)),
                 dropout=0.5):

        super(CNN, self).__init__()
        self.vocab_size, self.embed_dim = pretrained_embedding.shape
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
                                                        
        self.conv1d1 = nn.Conv1d(in_channels=self.embed_dim, # 300
                                 out_channels=100,
                                 kernel_size=3) 

        # Fully-connected layer and Dropout
        self.fc = nn.Linear(100, num_classes) ## ex-sum(num_filters)
        self.dropout = nn.Dropout(p=dropout) ## overfitting

    def forward(self, input_ids):

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

       
        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv = F.relu(self.conv1d1(x_reshaped)) 

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool = F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])

        ## remove second dimension 
        x_squz = x_pool.squeeze(dim=2)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_squz))
        
        return logits

In [12]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [13]:
import torch.optim as optim
import time 
import random
m = CNN(pretrained_embedding=emb,
                 freeze_embedding=False,
                 vocab_size=len(word2idx),
                 embed_dim=300,
                 num_classes=len(set(labels)),
                 dropout=0.5)

m.to(device)
loss_fn = nn.CrossEntropyLoss() 
o = optim.Adam(m.parameters(), lr=0.001)
best_accuracy = 0

for epoch_i in range(30):

    t0_epoch = time.time()
    total_loss = 0
   
    m.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        m.zero_grad()

        logits = m(b_input_ids)

        loss = loss_fn(logits, b_labels)
        total_loss += loss.item()
        loss.backward()

        o.step()

    # Calculate the average loss over the entire training data
    avg_train_loss = total_loss / len(train_dataloader)

    m.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = m(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)


    # Track the best accuracy
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy

    # Print performance over the entire training data
    time_elapsed = time.time() - t0_epoch
    print('epoch :', epoch_i, 'total loss: ', total_loss)
    print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")

epoch : 0 total loss:  92693.46712614033
   1    |   2.393139   |  2.282082  |   26.77   |  152.51  
epoch : 1 total loss:  91758.62284770737
   2    |   2.369004   |  2.285797  |   27.42   |  151.89  
epoch : 2 total loss:  92200.38801175753
   3    |   2.380409   |  2.320092  |   27.95   |  152.03  
epoch : 3 total loss:  92148.46855435043
   4    |   2.379069   |  2.302074  |   28.88   |  152.03  
epoch : 4 total loss:  91700.35843872729
   5    |   2.367500   |  2.409369  |   25.30   |  152.01  
epoch : 5 total loss:  91956.44772426567
   6    |   2.374111   |  2.386781  |   27.14   |  151.80  
epoch : 6 total loss:  92238.39589268557
   7    |   2.381390   |  2.375813  |   27.25   |  151.78  
epoch : 7 total loss:  93068.06209074115
   8    |   2.402811   |  2.438652  |   26.23   |  151.63  
epoch : 8 total loss:  93068.82366311371
   9    |   2.402830   |  2.553468  |   25.77   |  151.86  
epoch : 9 total loss:  92976.59273431583
  10    |   2.400449   |  2.669631  |   25.91   | 

In [14]:
torch.save(m.state_dict(), "model.pt")